# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [2]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x109a562e0>)

In [3]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [266]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [35]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
4839,4.1,1235.75,-200,21.275833,1331.50,641.0,600.50,152.0,1683.00,1549.50,17.800,45.674999,0.922429
1600,2.0,1052.50,-200,9.784034,969.50,138.0,930.00,105.0,1472.50,991.00,25.500,21.050000,0.677027
4299,1.6,987.75,-200,7.049952,859.00,-200.0,844.75,-200.0,1341.25,799.25,27.125,26.925000,0.952434
9227,0.5,900.75,-200,2.062928,588.75,122.0,933.00,69.9,1108.50,753.75,10.200,77.875000,0.969173
8581,1.4,1216.50,-200,6.742318,845.50,262.8,649.75,135.5,1120.25,1480.50,2.850,75.575001,0.574151


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [267]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [268]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [269]:
np.random.seed(65375)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [270]:
ndata, ncols = np.shape(air_quality)
col_names = air_quality.columns.values

def impute(df_unproc, ncols, col_names):
    mean_notnull = np.empty(ncols)
    df_proc = df_unproc.copy()
    for i in range(ncols):
        mean_notnull[i] = np.mean(df_proc[col_names[i]][df_proc[col_names[i]]!=-200])
        df_proc[col_names[i]][df_proc[col_names[i]]==-200] = mean_notnull[i]
    return df_proc

def save_mean_data(df_impute):
    return np.mean(df_impute)

def save_std_data(df_impute):
    return np.std(df_impute)

def standardise_data(df_impute, mean_data, std_data, col_names):
    df_proc = df_impute.copy()
    for i in range(1, ncols):
        df_proc[col_names[i]] = (df_proc[col_names[i]] - mean_data[i]) / std_data[i]
    return df_proc

#fix the missing values for training dataset
data_training_impute = impute(data_training_unproc, ncols, col_names)

#save the mean, std of data_training_impute
mean_data = save_mean_data(data_training_impute)
std_data = save_std_data(data_training_impute)

#standardlise the training dataset
data_training_proc = standardise_data(data_training_impute, mean_data, std_data, col_names)



## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [271]:
# Split the target and features
def split_target_features(data_proc):
    yTrain = (np.array(data_proc['CO(GT)']).reshape(-1, 1)).copy()
    XTrain = np.hstack((np.ones(yTrain.shape).reshape(-1, 1), np.array(data_proc.drop('CO(GT)', axis=1)))).copy()
    return XTrain, yTrain
XTrain, yTrain = split_target_features(data_training_proc)
#print(XTrain,'\n' ,yTrain)

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [272]:
def gradient_descent(Xdata, ydata, w, gama, ita, s):
    x = Xdata
    y = ydata
    w_new = np.empty(w.shape)
    w_old = w
    test = 0
    
    for i in range(200):
        w_new = (1 - ita*gama) * w_old # the first term on the rhs
        for j in range(s):
            temp = x[j].copy().reshape(-1,1)
            w_new += 2*ita/s * (y[j]*temp - np.dot(temp.T,w_old)*temp)
        w_old = w_new
    return w_new
gradient_descent(XTrain, yTrain,w, 0.001, 0.01, 400)

array([[ 2.12707133],
       [ 0.36699644],
       [ 0.47747072],
       [ 0.11514223],
       [ 0.50555582],
       [ 0.04928669],
       [ 0.20279588],
       [ 0.36608647],
       [-0.2970428 ],
       [-0.15706756],
       [-0.1515783 ],
       [-0.02158918]])

In [273]:
max_n_iter = 200 # max iterative procedure

# Replace the missing values and standardise the validation data
data_val_impute = impute(data_val_unproc, ncols, col_names)
mean_val_data = save_mean_data(data_val_impute)
std_val_data = save_std_data(data_val_impute)
data_val_proc = standardise_data(data_val_impute, mean_val_data, std_val_data, col_names)

#Create the grid of parameters
def set_gamas(min, max, num = 5):
    gamas = np.logspace(min, max, num = num)
    return gamas

def set_itas(min, max, num = 5):
    itas = np.logspace(min, max, num = num)
    return itas

def set_S(min, max, num = 5):
    S = np.linspace(min, max, num = num, dtype = int)
    return S

# Create XVal, yVal
XVal, yVal = split_target_features(data_val_proc)
# print('XVal:\n', XVal)
# print('yVal:\n', yVal)

##############
def gradient_descent(Xdata, ydata, w, gama, ita, s):
    x = Xdata
    y = ydata
    w_new = np.empty(w.shape)
    w_old = w
    test = 0
    
    for i in range(200):
        w_new = (1 - ita*gama) * w_old # the first term on the rhs
        for j in range(s):
            temp = x[j].copy().reshape(-1,1)
            w_new += 2*ita/s * (y[j]*temp - np.dot(temp.T,w_old)*temp)
        w_old = w_new
    return w_new

#find the best parameters
def find_better_parameters(gamas, itas, S, XTrain, yTrain,XTest, yTest):
    glen = len(gamas)
    ilen = len(itas)
    Slen = len(S)
    rmse_min = 100
    ita = itas[0]
    gama = gamas[0]
    s = S[0]
    for i in range(glen):
        for j in range(ilen):
            for k in range(Slen):
                temp = gradient_descent(XTrain, yTrain, w, gamas[i], itas[j], S[k])
                rmse = RMSE(temp, yTest, XTest)
                if rmse_min > rmse:
                    rmse_min = rmse
                    ita = itas[j]
                    gama = gamas[i]
                    s = S[k]
    return (rmse_min, ita, gama, s)

def RMSE(w, yTest, XTest):
    y_pred = np.dot(XTest,w)
    absError = y_pred - yTest
    error_test = np.sqrt(np.mean(np.square(absError)))
    return error_test
            
# create starting value of itas, gamas, S, w

itas = set_itas(-4, 0, num = 5)
gamas = set_gamas(-6, -1, num = 5)
S = set_S(200, 800, num = 5)
w = np.array(np.ones(XVal.shape[1])).reshape(-1,1)

# Compute the min(RSME) and find the best value of S, 𝜆, 𝜂
rmse_min, ita, gama, s = find_better_parameters(gamas, itas, S, XTrain, yTrain, XVal, yVal)
print('RMSE_min:', rmse_min, 'ita:', ita, 'gama:', gama, 's:', s)


<ipython-input-273-d359897bd94f>:67: RuntimeWarning: overflow encountered in square
  error_test = np.sqrt(np.mean(np.square(absError)))


RMSE_min: 0.4682172249683078 ita: 0.1 gama: 0.005623413251903491 s: 800


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [274]:
# combining the unproc_training data and  validation data
data_train_combine_unproc = air_quality.iloc[index[0:N+Nval], :].copy()
data_train_combine_impute = impute(data_train_combine_unproc, ncols, col_names)

#saving the mean and std data
train_combine_mean_data = save_mean_data(data_train_combine_impute)
train_combine_std_data = save_std_data(data_train_combine_impute)

#standardlising the combine data
data_train_combine_proc = standardise_data(data_train_combine_impute, train_combine_mean_data, train_combine_std_data, col_names)

#print(data_train_combine_proc)

### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [287]:
# Preprocess the test data by imputing the missing data and standardising it
data_test_impute = impute(data_test_unproc, ncols, col_names)

test_mean_data = save_mean_data(data_test_impute)
test_std_data = save_std_data(data_test_impute)

data_test_proc = standardise_data(data_test_impute, test_mean_data, test_std_data, col_names)


#split the X_train,y_train, x_test, y_test
X_train, y_train = split_target_features(data_train_combine_proc)
X_test, y_test = split_target_features(data_test_proc)

#begin to compute w and rmse
w = gradient_descent(X_train, y_train,w, gama, ita, s)
print('w:', w)
rmse = RMSE(w, y_test, X_test)
print('the RMSE over the test data: ', rmse)

w: [[ 2.14215137]
 [ 0.35075786]
 [ 0.47926837]
 [ 0.11954783]
 [ 0.50145882]
 [ 0.04118728]
 [ 0.19232418]
 [ 0.33241691]
 [-0.28105951]
 [-0.16904974]
 [-0.13068197]
 [-0.00738052]]
the RMSE over the test data:  0.4977296547216286


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [276]:
# Write your code here
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

preprocessing = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [283]:
#split the labels and features
X_train_unproc_q2 = (data_training_unproc.iloc[:, 1:12]).copy()
y_train_q2 = (data_training_unproc.iloc[:, 0]).copy()
X_val_unproc_q2 = (data_val_unproc.iloc[:, 1:12]).copy()
y_val_q2 = (data_val_unproc.iloc[:, 0]).copy()

X_train_q2 = preprocessing.fit_transform(X_train_unproc_q2)
X_val_q2 = preprocessing.transform(X_val_unproc_q2)
print(X_train_unproc_q2)
# print(X_val_q2)
# print(col_names)
# print(XTrain[0])

      PT08.S1(CO)   C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  \
8577       991.75   3.047943         655.00   -200.0        875.75   -200.0   
1599      1046.75   8.221421         908.25    119.0        912.50     89.0   
4777       986.75   9.823749         971.00    218.0        876.25     68.0   
494       1580.25  19.949354        1295.00    230.0        639.25    115.0   
151       1071.50   5.415775         784.00     95.0       1065.50     90.0   
...           ...        ...            ...      ...           ...      ...   
7319      1210.25   8.826223         932.50    348.2        647.75    156.4   
2544      1075.00  11.144589        1019.50    109.0        802.25    103.0   
8937       988.75   3.239489         666.75    167.8        824.25    117.3   
4690       877.75   3.843684         702.00     59.0        948.50     44.0   
6269      1502.00  18.555071        1255.50    696.0        559.50    154.0   

      PT08.S4(NO2)  PT08.S5(O3)          T         

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [284]:
whole_X_train_q2 = np.vstack((X_train_q2, X_val_q2))
whole_y_train_q2 = np.hstack((y_train_q2, y_val_q2))
print(whole_X_train_q2.shape)
print(whole_y_train_q2.shape)

(6523, 11)
(6523,)


In [285]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.ensemble import RandomForestRegressor

# Use PredefinedSplit to tell the cross validator which instances to use for
# training and which ones for validation

test_fold = np.zeros((np.shape(X_train_q2)[0] + np.shape(X_val_q2)[0], 1))
test_fold[0:np.shape(X_train_q2)[0]] = -1
ps = PredefinedSplit(test_fold)

# Concatenating the attributes and the labels
whole_X_train_q2 = np.vstack((X_train_q2, X_val_q2))
whole_y_train_q2 = np.hstack((y_train_q2, y_val_q2))
#Create a grid of values to explore parameters
n_estimators_q2 = [20, 50, 100, 200, 300]
max_samples_q2 =[500, 1000, 2000, 3000, 4000]
max_features_q2 = [3, 5, 7, 9, 11]
param_grid_q2 = dict(n_estimators = n_estimators_q2, max_samples = max_samples_q2, max_features = max_features_q2)
grid_regression_q2 = GridSearchCV(RandomForestRegressor(), param_grid=param_grid_q2, cv=ps, scoring='neg_mean_squared_error')

grid_regression_q2.fit(whole_X_train_q2, whole_y_train_q2)
print('Best values in my grid:', grid_regression_q2.best_params_)

Best values in my grid: {'max_features': 11, 'max_samples': 4000, 'n_estimators': 200}


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [290]:
from sklearn.metrics import mean_squared_error
data_train_all_unproc_q2 = air_quality.iloc[index[0:N+Nval], :].copy()
data_train_attributes_unproc = data_train_all_unproc_q2.iloc[:, 1:12].copy()
data_train_labels = data_train_all_unproc_q2.iloc[:,0].copy()

data_test_attributes_unproc = data_test_unproc.iloc[:, 1:12].copy()
data_test_labels = data_test_unproc.iloc[:, 0].copy()

data_train_attributes_q2 = preprocessing.fit_transform(data_train_attributes_unproc)
data_test_attributes_q2 = preprocessing.transform(data_test_attributes_unproc)

regr_q2 = RandomForestRegressor(n_estimators=grid_regression_q2.best_params_["n_estimators"], max_samples=grid_regression_q2.best_params_["max_samples"], max_features=grid_regression_q2.best_params_["max_features"])
regr_q2.fit(data_train_attributes_q2, data_train_labels)
predictions_q2 = regr_q2.predict(data_test_attributes_q2)
error_mod_q2 = np.sqrt(mean_squared_error(data_test_labels, predictions_q2))
print('The RMSE on the test set data is:', error_mod_q2)

The RMSE on the test set data is: 0.42782154347438534
